# Colab setup

In [ ]:
import sys
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Projetos/ZAAI
sys.path.append('/content/drive/My Drive/Projetos/ZAAI/')
%pip install -r requirements.txt

Mounted at /content/drive
/content/drive/My Drive/Projetos/ZAAI
  Cloning https://github.com/amazon-science/chronos-forecasting.git to /tmp/pip-req-build-r7qz7dn2
  Running command git clone --filter=blob:none --quiet https://github.com/amazon-science/chronos-forecasting.git /tmp/pip-req-build-r7qz7dn2


In [ ]:
# import TiDE_tuning

# Load Libraries and set global variables

In [ ]:
%load_ext autoreload
%autoreload 2
from darts.dataprocessing.transformers import StaticCovariatesTransformer, MissingValuesFiller
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.utils.likelihood_models import QuantileRegression
from darts.dataprocessing.transformers import Scaler
from darts.dataprocessing.pipeline import Pipeline
from chronos import ChronosPipeline
from darts.models import TiDEModel
from darts import TimeSeries
import pandas as pd
import numpy as np
import random
import torch
import utils

TIME_COL = "Date"
TARGET = "Weekly_Sales"
RES_TARGET = "residuals"
STATIC_COV = ["Store", "Dept", "Type", "Size"]
DYNAMIC_COV_FILL_0 = ["IsHoliday", 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
DYNAMIC_COV_FILL_INTERPOLATE = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment']
FREQ = "W-FRI"

SCALER = Scaler()
TRANSFORMER = StaticCovariatesTransformer()
PIPELINE = Pipeline([SCALER, TRANSFORMER])

FORECAST_HORIZON = 10 # number of weeks to forecast
TOP_STORES = 500 # number of top stores to forecast

CHRONOS_ARCHITECTURE = ("amazon/chronos-t5-tiny", "cpu")
# CHRONOS_ARCHITECTURE = ("amazon/chronos-t5-large","cuda")
# CHRONOS_ARCHITECTURE = ("amazon/chronos-t5-tiny","cuda")

# Load Datasets for TiDE and Chronos sales forecast

In [ ]:
# load data and exogenous features
df = pd.read_csv('data/train.csv')
store_info = pd.read_csv('data/stores.csv')
exo_feat = pd.read_csv('data/features.csv').drop(columns='IsHoliday')

# join all data frames
df = pd.merge(df, store_info, on=['Store'], how='left')
df = pd.merge(df, exo_feat, on=['Store', TIME_COL], how='left')

# create unique id
df["unique_id"] = df['Store'].astype(str)+'-'+df['Dept'].astype(str)

print(f"Distinct number of time series: {len(df['unique_id'].unique())}")
df

## Pre-process dataset

In [ ]:
df[TIME_COL] = pd.to_datetime(df[TIME_COL])
df[TARGET] = np.where(df[TARGET] < 0, 0, df[TARGET]) # remove negative values
df[['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4','MarkDown5']] = df[['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4','MarkDown5']].fillna(0) # fill missing values with nan
df["IsHoliday"] = df["IsHoliday"]*1 # convert boolean into binary
df["Size"] = np.where(df["Size"] < store_info["Size"].quantile(0.25), "small",
                np.where(df["Size"] > store_info["Size"].quantile(0.75), "large",
                "medium")) # make size a categorical variable

top_stores = df.groupby(['unique_id']).agg({TARGET: 'sum'}).reset_index().sort_values(by=TARGET, ascending=False).head(TOP_STORES)
df = df[df['unique_id'].isin(top_stores['unique_id'])]

print(f"Distinct number of time series: {len(df['unique_id'].unique())}")
df

# Training with the whole dataset until the start of the window, and forecast for each window

In [ ]:
window1_start=pd.to_datetime('2012-01-20')
window1=(window1_start,window1_start + pd.Timedelta(weeks=FORECAST_HORIZON))
test1 = df[(df[TIME_COL] <= window1[1])]

window2_start = pd.to_datetime('2012-03-16')
window2=(window2_start,window2_start + pd.Timedelta(weeks=FORECAST_HORIZON))
test2 = df[(df[TIME_COL] <= window2[1])]

window3_start = pd.to_datetime('2012-05-25')
window3=(window3_start,window3_start + pd.Timedelta(weeks=FORECAST_HORIZON))
test3 = df[(df[TIME_COL] <= window3[1])]

window4_start = pd.to_datetime('2012-08-03')
window4=(window4_start,window4_start + pd.Timedelta(weeks=FORECAST_HORIZON))
test4 = df[(df[TIME_COL] <= window4[1])]

window = (window1[0],window1[1])
#window = (window2[0],window2[1])
#window = (window3[0],window3[1])
#window = (window4[0],window4[1])

windows = [window1, window2, window3, window4]

## Split Data

In [ ]:
train = df[(df["Date"] <= window[0])]
test = df[(df["Date"] > window[0]) & (df["Date"] <= window[1])]
df = df[(df[TIME_COL] <= window[1])]

train_darts = TimeSeries.from_group_dataframe(
    df=train,
    group_cols=STATIC_COV,
    time_col=TIME_COL,
    value_cols=TARGET,
    freq=FREQ,
    fill_missing_dates=True,
    fillna_value=0)

print(f"Weeks for training: {len(train[TIME_COL].unique())} from {min(train[TIME_COL]).date()} to {max(train[TIME_COL]).date()}")
print(f"Weeks for testing: {len(test[TIME_COL].unique())} from {min(test[TIME_COL]).date()} to {max(test[TIME_COL]).date()}")

# TiDE

In [ ]:
tide_forecast_window1 = pd.read_csv('data/tide_forecast_2012-01-20|2012-03-30.csv')
tide_forecast_window1['Date'] = pd.to_datetime(tide_forecast_window1['Date'])

tide_forecast_window2 = pd.read_csv('data/tide_forecast_2012-03-16|2012-05-25.csv')
tide_forecast_window2['Date'] = pd.to_datetime(tide_forecast_window2['Date'])

tide_forecast_window3 = pd.read_csv('data/tide_forecast_2012-05-25|2012-08-03.csv')
tide_forecast_window3['Date'] = pd.to_datetime(tide_forecast_window3['Date'])

tide_forecast_window4 = pd.read_csv('data/tide_forecast_2012-08-03|2012-10-12.csv')
tide_forecast_window4['Date'] = pd.to_datetime(tide_forecast_window4['Date'])

tide_forecast = [tide_forecast_window1, tide_forecast_window2, tide_forecast_window3, tide_forecast_window4]

In [ ]:
'''
# create dynamic covariates for each serie in the training darts
dynamic_covariates = []
for serie in train_darts:
    # add the month and week as a covariate
    covariate = datetime_attribute_timeseries(
        serie,
        attribute="month",
        one_hot=True,
        cyclic=False,
        add_length=FORECAST_HORIZON,
    )
    covariate = covariate.stack(
        datetime_attribute_timeseries(
            serie,
            attribute="week",
            one_hot=True,
            cyclic=False,
            add_length=FORECAST_HORIZON,
        )
    )

    store = serie.static_covariates['Store'].item()
    dept = serie.static_covariates['Dept'].item()

    # create covariates to fill with 0
    covariate = covariate.stack(
                TimeSeries.from_dataframe(df[(df['Store'] == store) & (df['Dept'] == dept)], time_col=TIME_COL, value_cols=DYNAMIC_COV_FILL_0, freq=FREQ, fill_missing_dates=True, fillna_value=0)
            )

    # create covariates to fill with interpolation
    dyn_cov_interp = TimeSeries.from_dataframe(df[(df['Store'] == store) & (df['Dept'] == dept)], time_col=TIME_COL, value_cols=DYNAMIC_COV_FILL_INTERPOLATE, freq=FREQ, fill_missing_dates=True)
    covariate = covariate.stack(MissingValuesFiller().transform(dyn_cov_interp))

    dynamic_covariates.append(covariate)
'''

In [ ]:
'''
# scale covariates
dynamic_covariates_transformed = SCALER.fit_transform(dynamic_covariates)

# scale data and transform static covariates
data_transformed = PIPELINE.fit_transform(train_darts)

TiDE_params = {
    "input_chunk_length": len(train[TIME_COL].unique()) - FORECAST_HORIZON, # number of weeks to lookback
    "output_chunk_length": FORECAST_HORIZON, # number of weeks to forecast
    "num_encoder_layers": 2,
    "num_decoder_layers": 2,
    "decoder_output_dim": 1,
    "hidden_size": 15,
    "temporal_width_past": 4,
    "temporal_width_future": 4,
    "temporal_decoder_hidden": 26,
    "dropout": 0.1,
    "batch_size": 16,
    "n_epochs": 50,
    "likelihood": QuantileRegression(quantiles=[0.25, 0.5, 0.75]),
    "random_state": 42,
    "use_static_covariates": True,
    "optimizer_kwargs": {"lr": 1e-3},
    "use_reversible_instance_norm": False,
}

model = TiDEModel(**TiDE_params)
model.fit(data_transformed, future_covariates=dynamic_covariates_transformed, verbose=False)
pred = PIPELINE.inverse_transform(model.predict(n=FORECAST_HORIZON, series=data_transformed, future_covariates=dynamic_covariates_transformed, num_samples=50))
tide_forecast = utils.transform_predictions_to_pandas(pred, TARGET, train_darts, [0.25, 0.5, 0.75])
tide_forecast.to_csv('data/tide_forecast.csv' + str(window[0].date().strftime('%Y-%m-%d')) + '|' + str(
        window[1].date().strftime('%Y-%m-%d')) + '.csv', index=False)
tide_forecast
'''

# Chronos

In [ ]:
chronos_forecast_window1 = pd.read_csv('data/chronos_forecast_2012-01-20|2012-03-30.csv')
chronos_forecast_window1['Date'] = pd.to_datetime(chronos_forecast_window1['Date'])

chronos_forecast_window2 = pd.read_csv('data/chronos_forecast_2012-03-16|2012-05-25.csv')
chronos_forecast_window2['Date'] = pd.to_datetime(chronos_forecast_window2['Date'])

chronos_forecast_window3 = pd.read_csv('data/chronos_forecast_2012-05-25|2012-08-03.csv')
chronos_forecast_window3['Date'] = pd.to_datetime(chronos_forecast_window3['Date'])

chronos_forecast_window4 = pd.read_csv('data/chronos_forecast_2012-08-03|2012-10-12.csv')
chronos_forecast_window4['Date'] = pd.to_datetime(chronos_forecast_window4['Date'])

chronos_forecasts = [chronos_forecast_window1, chronos_forecast_window2, chronos_forecast_window3, chronos_forecast_window4]

In [ ]:
'''
# Load the Chronos pipeline
pipeline = ChronosPipeline.from_pretrained(
    CHRONOS_ARCHITECTURE[0],
    device_map=CHRONOS_ARCHITECTURE[1],
    torch_dtype=torch.bfloat16)

forecast = []
for ts in train_darts:
    # Forecast
    lower, mid, upper = utils.chronos_forecast(pipeline, ts.pd_dataframe().reset_index(), FORECAST_HORIZON)
    forecast.append(utils.convert_forecast_to_pandas([lower, mid, upper], test[test['unique_id'] == str(int(list(ts.static_covariates_values())[0][0]))+'-'+str(int(list(ts.static_covariates_values())[0][1]))]))
# Convert list to data frames
chronos_forecast = pd.concat(forecast)
chronos_forecast.to_csv('data/chronos_forecast.csv' + str(window[0].date().strftime('%Y-%m-%d')) + '|' + str(
        window[1].date().strftime('%Y-%m-%d')) + '.csv', index=False)
chronos_forecast
'''

# Now let's combine the forecasts from Chronos and residuals forecast from TiDE

In [ ]:
'''
final_forecast_window1 = pd.read_csv('data/final_forecast_2012-01-20|2012-03-30.csv')
final_forecast_window1['Date'] = pd.to_datetime(final_forecast_window1['Date'])

final_forecast_window2 = pd.read_csv('data/final_forecast_2012-03-16|2012-05-25.csv')
final_forecast_window2['Date'] = pd.to_datetime(final_forecast_window2['Date'])

final_forecast_window3 = pd.read_csv('data/final_forecast_2012-05-25|2012-08-03.csv')
final_forecast_window3['Date'] = pd.to_datetime(final_forecast_window3['Date'])

final_forecast_window4 = pd.read_csv('data/final_forecast_2012-08-03|2012-10-12.csv')
final_forecast_window4['Date'] = pd.to_datetime(final_forecast_window4['Date'])

final_forecast = [final_forecast_window1, final_forecast_window2, final_forecast_window3, final_forecast_window4]
'''

## TiDE residuals forecast

In [ ]:
final_forecast = []

for window, chronos_forecast in zip(windows, chronos_forecasts):
    residuals = pd.read_csv('data/residuals.csv')
    residuals[TIME_COL] = pd.to_datetime(residuals[TIME_COL])
    residuals[['Store', 'Dept']] = residuals['unique_id'].str.split('-', expand=True).astype(int)

    residuals_train = residuals[residuals[TIME_COL] <= window[0]]
    residuals = residuals[(residuals[TIME_COL] <= window[1])]

    residuals_darts = TimeSeries.from_group_dataframe(
        df=residuals_train,
        group_cols=STATIC_COV,
        time_col=TIME_COL,
        value_cols=RES_TARGET,
        freq=FREQ,
        fill_missing_dates=True,
        fillna_value=0)

    print(f"Weeks for training: {len(residuals_train[TIME_COL].unique())} from {min(residuals_train[TIME_COL]).date()} to {max(residuals_train[TIME_COL]).date()}")

    # create dynamic covariates for each serie in the training darts
    dynamic_covariates = []
    for serie in residuals_darts:
        # add the month and week as a covariate
        covariate = datetime_attribute_timeseries(
            serie,
            attribute="month",
            one_hot=True,
            cyclic=False,
            add_length=FORECAST_HORIZON,
        )
        covariate = covariate.stack(
            datetime_attribute_timeseries(
                serie,
                attribute="week",
                one_hot=True,
                cyclic=False,
                add_length=FORECAST_HORIZON,
            )
        )

        store = serie.static_covariates['Store'].item()
        dept = serie.static_covariates['Dept'].item()

        # create covariates to fill with 0
        covariate = covariate.stack(
                    TimeSeries.from_dataframe(residuals[(residuals['Store'] == store) & (residuals['Dept'] == dept)], time_col=TIME_COL, value_cols=DYNAMIC_COV_FILL_0, freq=FREQ, fill_missing_dates=True, fillna_value=0)
                )

        # create covariates to fill with interpolation
        dyn_cov_interp = TimeSeries.from_dataframe(residuals[(residuals['Store'] == store) & (residuals['Dept'] == dept)], time_col=TIME_COL, value_cols=DYNAMIC_COV_FILL_INTERPOLATE, freq=FREQ, fill_missing_dates=True)
        covariate = covariate.stack(MissingValuesFiller().transform(dyn_cov_interp))

        dynamic_covariates.append(covariate)

    # scale covariates
    dynamic_covariates_transformed = SCALER.fit_transform(dynamic_covariates)

    # scale data and transform static covariates
    data_transformed = PIPELINE.fit_transform(residuals_darts)

    TiDE_params = {
        "input_chunk_length": len(residuals_train[TIME_COL].unique()) - FORECAST_HORIZON, # number of weeks to lookback
        "output_chunk_length": FORECAST_HORIZON,
        "num_encoder_layers": 3,
        "num_decoder_layers": 2,
        "decoder_output_dim": 1,
        "hidden_size": 15,
        "temporal_width_past": 4,
        "temporal_width_future": 4,
        "temporal_decoder_hidden": 26,
        "dropout": 0.1,
        "batch_size": 16,
        "n_epochs": 50,
        "likelihood": QuantileRegression(quantiles=[0.25, 0.5, 0.75]),
        "random_state": 42,
        "use_static_covariates": True,
        "optimizer_kwargs": {"lr": 1e-05},
        "use_reversible_instance_norm": False,
    }

    model = TiDEModel(**TiDE_params)
    model.fit(data_transformed, future_covariates=dynamic_covariates_transformed, verbose=False)
    pred = PIPELINE.inverse_transform(model.predict(n=FORECAST_HORIZON, series=data_transformed, future_covariates=dynamic_covariates_transformed, num_samples=50))
    residuals_forecast = utils.transform_predictions_to_pandas(pred, RES_TARGET, residuals_darts, [0.25, 0.5, 0.75], convert=False)

    # Concatenate the two dataframes
    combined_df = pd.concat([chronos_forecast, residuals_forecast])

    # Group by 'unique_id' and 'Date' and sum the forecast values
    final_forecast = combined_df.groupby(['unique_id', 'Date']).agg({
        'forecast_lower': 'sum',
        'forecast': 'sum',
        'forecast_upper': 'sum'
    }).reset_index()

    #final_forecast.to_csv('data/final_forecast_' + str(window[0].date().strftime('%Y-%m-%d')) + '|' + str(
    #        window[1].date().strftime('%Y-%m-%d')) + '.csv', index=False)

    final_forecast.append(final_forecast)

## Train and predict

# Plot Actuals and Forecast

In [ ]:
# get series ordered by volume in a descending way
series = test.groupby('unique_id')[TARGET].sum().reset_index().sort_values(by=TARGET, ascending=False)['unique_id'].tolist()

tests = [test1,test2,test3,test4]

for f, c, t, te in zip(final_forecast, chronos_forecasts, tide_forecast, tests):
    for ts in series[:1]:
        forecasts = [(f[f["unique_id"] == ts],"Chronos + TiDE"),
                     (c[c["unique_id"] == ts],"Chronos"),
                     (t[t["unique_id"] == ts],"TiDE")]

        utils.plot_multiple_forecasts(actuals_data=te[te["unique_id"]==ts],
                                      forecast_data_list=forecasts,
                                      title="Actuals vs Forecast",
                                      y_label="Weekly Sales",
                                      x_label="Date",
                                      forecast_horizon=FORECAST_HORIZON,
                                      interval=False)

## Evaluate forecast

In [ ]:
final_forecast_window1, final_forecast_window2, final_forecast_window3, final_forecast_window4 = final_forecast

mapes = [(utils.evaluation_metrics(chronos_forecast_window1, test1),
          utils.evaluation_metrics(tide_forecast_window1, test1),
          utils.evaluation_metrics(final_forecast_window1, test1)),

         (utils.evaluation_metrics(chronos_forecast_window2, test2),
          utils.evaluation_metrics(tide_forecast_window2, test2),
          utils.evaluation_metrics(final_forecast_window2, test2)),

         (utils.evaluation_metrics(chronos_forecast_window3, test3),
          utils.evaluation_metrics(tide_forecast_window3, test3),
          utils.evaluation_metrics(final_forecast_window3, test3)),

         (utils.evaluation_metrics(chronos_forecast_window4, test4),
          utils.evaluation_metrics(tide_forecast_window4, test4),
          utils.evaluation_metrics(final_forecast_window4, test4))
         ]

utils.plot_multiple_model_comparison(windows, mapes)